[View in Colaboratory](https://colab.research.google.com/github/wonwooddo/Pytorch_tutorial/blob/master/01_basic_neuralnetwork.ipynb)

In [1]:
!pip install torch
!pip install torchvision

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [0]:
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [12]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

Processing...
Done!


In [0]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [0]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)


In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [22]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.4971
Epoch [1/5], Step [200/600], Loss: 0.3898
Epoch [1/5], Step [300/600], Loss: 0.1831
Epoch [1/5], Step [400/600], Loss: 0.2858
Epoch [1/5], Step [500/600], Loss: 0.2533
Epoch [1/5], Step [600/600], Loss: 0.1095
Epoch [2/5], Step [100/600], Loss: 0.1723
Epoch [2/5], Step [200/600], Loss: 0.2185
Epoch [2/5], Step [300/600], Loss: 0.0753
Epoch [2/5], Step [400/600], Loss: 0.0676
Epoch [2/5], Step [500/600], Loss: 0.0703
Epoch [2/5], Step [600/600], Loss: 0.1050
Epoch [3/5], Step [100/600], Loss: 0.2245
Epoch [3/5], Step [200/600], Loss: 0.0929
Epoch [3/5], Step [300/600], Loss: 0.0767
Epoch [3/5], Step [400/600], Loss: 0.1275
Epoch [3/5], Step [500/600], Loss: 0.0524
Epoch [3/5], Step [600/600], Loss: 0.1052
Epoch [4/5], Step [100/600], Loss: 0.0351
Epoch [4/5], Step [200/600], Loss: 0.0555
Epoch [4/5], Step [300/600], Loss: 0.0310
Epoch [4/5], Step [400/600], Loss: 0.0165
Epoch [4/5], Step [500/600], Loss: 0.0533
Epoch [4/5], Step [600/600], Loss:

In [24]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')


Accuracy of the network on the 10000 test images: 97 %
